## Exercise 02. Join ##

## Imports ##

In [24]:
import pandas as pd
import sqlite3

## Task ##

## Create datamart table ##
1. Создайте подключение к базе данных с использованием библиотеки `sqlite3`.
2. Создайте новую таблицу с именем `datamart` в базе данных, объединив таблицы `pageviews` и `checker` с помощью одного запроса.
   - Таблица должна содержать следующие столбцы: "uid", "labname", "first_commit_ts" и "first_view_ts".
   - "first_commit_ts" — новое имя для столбца "timestamp" из таблицы checker. Показывает первый коммит от конкретной лаборатории и пользователя.
   - "first_view_ts" показывает время первого посещения пользователем таблицы pageviews. Это временная метка посещения новостной ленты пользователем.
   - Фильтр `status = 'ready'` должен сохраняться.
   - Фильтр `numTrials = 1` должен сохраняться.
   - "labnames" должны быть из списка: `laba04`, `laba04s`, `laba05`, `laba06`, `laba06s` и `project1`.
   - Таблица должна содержать только пользователей (uids, начинающиеся с user_*), без администраторов.
   - "first_commit_ts" и "first_view_ts" должны быть преобразованы в формат `datetime64[ns]`.

In [25]:
conn = sqlite3.connect('../data/checking-logs.sqlite')

datamart = pd.read_sql('SELECT t1.uid, t1.labname, t1.timestamp, p.first_view_ts FROM (SELECT * FROM checker WHERE status="ready" AND numTrials=1 AND labname IN ("laba04", "laba04s", "laba05", "laba06", "laba06s", "project1") AND uid LIKE "%user%") AS t1 LEFT JOIN(SELECT uid, MIN(datetime) AS first_view_ts FROM pageviews GROUP BY uid) p ON t1.uid = p.uid;', conn)
datamart = datamart[['uid', 'labname', 'timestamp', 'first_view_ts']]
datamart = datamart.rename(columns={'timestamp': 'first_commit_ts'})
datamart['first_commit_ts'] = pd.to_datetime(datamart['first_commit_ts'])
datamart['first_view_ts'] = pd.to_datetime(datamart['first_view_ts'])
datamart.to_sql('datamart', conn, if_exists='replace', index=False)
datamart.head()
datamart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              140 non-null    object        
 1   labname          140 non-null    object        
 2   first_commit_ts  140 non-null    datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 4.5+ KB


## New dataframes and tables ##
3. Используя методы Pandas, создайте два датафрейма: `test` и `control`.
   - `test` должен содержать пользователей с заполненными значениями в "first_view_ts".
   - `control` должен содержать пользователей с отсутствующими значениями в "first_view_ts".
   - Замените отсутствующие значения в таблице control на среднее значение "first_view_ts" из пользователей test. Это значение будет использоваться для дальнейшего анализа.
   - Сохраните обе таблицы в базе данных. Они понадобятся в следующих заданиях.
4. Закройте соединение.

In [26]:
test = datamart[datamart['first_view_ts'].isna() == False].copy()
test
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59 entries, 3 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              59 non-null     object        
 1   labname          59 non-null     object        
 2   first_commit_ts  59 non-null     datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 2.3+ KB


In [27]:
control = datamart[datamart['first_view_ts'].isna() == True].copy()
control
control.info()

<class 'pandas.core.frame.DataFrame'>
Index: 81 entries, 0 to 137
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              81 non-null     object        
 1   labname          81 non-null     object        
 2   first_commit_ts  81 non-null     datetime64[ns]
 3   first_view_ts    0 non-null      datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 3.2+ KB


In [28]:
control['first_view_ts'] = test['first_view_ts'].mean()
control
control.info()

<class 'pandas.core.frame.DataFrame'>
Index: 81 entries, 0 to 137
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              81 non-null     object        
 1   labname          81 non-null     object        
 2   first_commit_ts  81 non-null     datetime64[ns]
 3   first_view_ts    81 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 3.2+ KB


In [29]:
test.to_sql('test', conn, if_exists='replace', index=False)
control.to_sql('control', conn, if_exists='replace', index=False)

81

In [30]:
conn.close()